In [ ]:
# pip install -qU langchain \
#     langchain-community \
#     langchain-together \
#     langchain-core \
#     langchain-google-vertexai \
#     gpt4all \
#     faiss-cpu \
#     faiss-gpu \
#     langgraph \
#     chromadb \
#     sentence-transformers \
#     tavily-python \
#     gradio

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders.web_base import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import nest_asyncio
from langchain_community.vectorstores.chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_together import ChatTogether
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate
from typing import Literal
from langchain.chains.combine_documents import stuff
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from IPython.core.display import Markdown
import json
import re

from langchain_core.runnables import (
    RunnableParallel,
    RunnableBranch,
    RunnablePassthrough,
)

from langchain_google_vertexai import ChatVertexAI

from langchain_core.messages import HumanMessage, AIMessage
import asyncio
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel , Field , validator


USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\Alchemist\Desktop\LangChainProject\venv\Lib\site-packages\IPython\core\interactiveshell.py:3670: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# import os, getpass
# print("Enter your Tavily API Key")
# TAVILY_API_KEY = getpass.getpass()
# print("Enter your Together API Key")
# TOGETHER_API_KEY = getpass.getpass()

# os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
# os.environ["TOGETHER_API_KEY"] = TOGETHER_API_KEY

In [ ]:
urls = [
    "https://www.webmd.com/a-to-z-guides/malaria",
    "https://www.webmd.com/diabetes/type-1-diabetes",
    "https://www.webmd.com/diabetes/type-2-diabetes",
    "https://www.webmd.com/migraines-headaches/migraines-headaches-migraines",
]

loader = WebBaseLoader(urls, bs_get_text_kwargs={"strip": True})
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
chunks = text_splitter.split_documents(docs)

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [10]:
embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2',show_progress=True, multi_process=True)
embedding_function

HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=True, show_progress=True)

In [11]:
vector_store = FAISS.from_documents(documents=chunks, embedding=embedding_function)

In [13]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002FD3C81AA20>, search_kwargs={'k': 3})